## The Notebook on Controlling the Thymio

For this project and for the control of the Thymio I had the vision of a wrapper for the node and the client that would take care of everything in a very modular way. I finally opted to create two classes one that is called AsyncClientInterface which is more of a lower level class and then I built another more semantic wraper of AsyncClientInterface called ThymioRobot.

### AsyncClientInterface

In [ ]:
from typing import Callable, Any
import dependencies.constants_robot as cst
from tdmclient import ClientAsync, aw


class InvalidArgumentError(Exception):
    pass


class AsyncClientInterface:
    """
    This is a class aimed to make the handling of the node easier and more readable
    """
    LEFT_MOTOR = "motor.left.target"
    RIGHT_MOTOR = "motor.right.target"
    RIGHT_SPEED = "motor.right.speed"
    LEFT_SPEED = "motor.left.speed"
    PROX_HORIZONTAL_VALUES = "prox.horizontal"
    LEDS_BOTTOM_LEFT = "leds.bottom.left"
    LEDS_BOTTOM_RIGHT = "leds.bottom.right"
    PROX_GROUND_DELTA = "prox.ground.delta"
    PROX_GROUND_REFLECTED = "prox.ground.reflected"
    MIC_INTENSITY = "mic.intensity"
    LEDS_TOP = "leds.top"
    __right_motor_value: int = 0
    __left_motor_value: int = 0
    _delta_calib = 1
    _refl_calib = 1

    def __init__(self, delta_calib: float = cst.DELTA_ROBOT, refl_calib: float = cst.REFL_ROBOT):
        """
        :param delta_calib: input known calibration for me it is 1.35
        :param refl_calib: input known calibration for me it is also about 1.35
        :param time_to_turn_const: time in seconds to turn 360° with motor speed = 150
        :type time_to_turn_const: float
        """
        self.client = ClientAsync()
        self.node = aw(self.client.wait_for_node())
        aw(self.node.lock())
        # Code to desactivate leds on start
        aw(self.node.compile("""call leds.prox.v(0, 0)\ncall leds.prox.h(0, 0, 0, 0, 0, 0, 0, 0)\ncall leds.temperature(0, 0)\ncall leds.top(0, 0, 0)\ncall leds.circle(0, 0, 0, 0, 0, 0, 0, 0)\ncall leds.buttons(0, 0, 0, 0)\ncall leds.sound(0)\ncall leds.rc(0)"""))
        self._refl_calib = refl_calib
        self._delta_calib = delta_calib


    def sleep(self, seconds):
        aw(self.client.sleep(seconds))

    def get_refl_calib(self) -> float:
        return self._refl_calib

    def get_del_calib(self) -> float:
        return self._delta_calib

    def set_motors(self, *args, **kwargs) -> None:
        """
        A flexible function that lets you input the speed of the motors

        :param args: left_motor then right_motor
        :type args: int
        :param kwargs: can put in right_motor or left_motor value in directly or put in array with length 2
        :type kwargs: int or list[int, int]

        :return: None
        """
        if len(args) == 2:
            aw(self.node.set_variables({
                self.LEFT_MOTOR: [args[0]],
                self.RIGHT_MOTOR: [args[1]]
            }))

        elif len(args) == 1:
            aw(self.node.set_variables({
                self.LEFT_MOTOR: [args[0]]
            }))

        if 'left_motor' in kwargs:
            if isinstance(kwargs['left_motor'], int):
                aw(self.node.set_variables({
                    self.LEFT_MOTOR: [kwargs['left_motor']]
                }))
            else:
                raise TypeError("value attributed to left_motor was not int")

        if 'right_motor' in kwargs:
            if isinstance(kwargs['right_motor'], int):
                aw(self.node.set_variables({
                    self.RIGHT_MOTOR: [kwargs['right_motor']]
                }))
            else:
                raise TypeError("value attributed to right_motor was not int")
        if 'motor' in kwargs:
            if isinstance(kwargs['motor'], list) and len(kwargs['motor']) == 2:
                aw(self.node.set_variables({
                    self.LEFT_MOTOR: [int(kwargs['motor'][0])],
                    self.RIGHT_MOTOR: [int(kwargs['motor'][1])]
                }))
            else:
                raise TypeError("either array too long or too short or did not respect type list")

    def calibrate_sensor_prox_ref(self, delta: bool, set_val: bool = True, preset_Val: float = None) -> float | bool:
        """
            A function that returns an int (sensor_1/sensor_2) used to calibrate the two sensors of
            prox_ground_reflected

            :param set_val: defines whether you would like to set the internal calib val
            :type set_val: bool

            :param delta: if function should be used for delta or reflected
            :type delta: bool

            :returns: ref_sensor_1/ref_sensor_2
            :rtype: float or bool
        """

        ref_value = self.get_sensor("prox_del") if delta else self.get_sensor("prox_ref")
        calibration = ref_value[0] / ref_value[1]
        if preset_Val:
            if delta:
                self._delta_calib = preset_Val
            else:
                self._refl_calib = preset_Val
            return preset_Val
        elif ref_value:
            if set_val:
                if delta:
                    self._delta_calib = calibration
                else:
                    self._refl_calib = calibration
            return calibration
        else:
            return False

    def set_led(self, sensor: str, value: int | list[int]) -> None:
        """
        A function that enables you to set sensors through the node.set_variables method the leds expect values between 0 and 32
        :param sensor: input your sensor
            - 'leds_top': expects three rgb values

        :param value: a list or an array containing the values you want to output
        """

        if sensor == "leds_top":
            if isinstance(value, list) and len(value) == 3:
                aw(self.node.wait_for_variables({"leds.top"}))
                for index, rgb in enumerate(value):
                    self.node.v.leds.top[index] = rgb
            else:
                raise TypeError("either did not input list or list was not of right length")

        self.node.flush()


    def get_sensor(self, sensor: str, calibrated: bool = False) -> list[int] | int | bool:
        """
        This is a function that returns the value of the sensor it accepts only a string and uses a switch case method


        :param sensor : Accepts  a limited number of strings\
            - 'prox_del' : this presents the array of prox ground delta with the option of calibration\
            - 'prox_ref' : this presents the array of prox ground reflected with the option of calibration\
            - self.CONSTANT: this is presented in the example in the description

        :type sensor: str

        :return: returns the value or array of a sensor, or returns False if invalid string is passed as argument
        :rtype: list[int] or int or bool or list[bool]

        Example:

        >>>Client = AsyncClientInterface(1.35,1.35)

        >>>print(Client.get_sensor(Client.PROX_GROUND_DELTA))
        """
        if sensor == "prox_del":
            aw(self.node.wait_for_variables({self.PROX_GROUND_DELTA}))
            if calibrated:
                calibrated_sensor_2 = int(list(self.node.v.prox.ground.delta)[1]) * self._delta_calib
                return [int(list(self.node.v.prox.ground.delta)[0]), int(calibrated_sensor_2)].copy()
            else:
                return list(self.node.v.prox.ground.delta).copy()

        elif sensor == "prox_ref":
            aw(self.node.wait_for_variables({self.PROX_GROUND_REFLECTED}))
            if calibrated:
                calibrated_sensor_2 = int(list(self.node.v.prox.ground.reflected)[1]) * self._refl_calib
                return [int(list(self.node.v.prox.ground.reflected)[0]), int(calibrated_sensor_2)].copy()
            else:
                return list(self.node.v.prox.ground.reflected).copy()

        else:
            aw(self.node.wait_for_variables({sensor}))
            attributes = sensor.split(".")
            node_with_attr = getattr(self.node, "v")
            for attribute in attributes:
                node_with_attr = getattr(node_with_attr, attribute)

            try:
                if not type(node_with_attr) == int:
                    return list(node_with_attr)
                else:
                    return node_with_attr
            except AttributeError:
                raise InvalidArgumentError("attribute was not found due to wrong input being put in")



    def __del__(self):
        aw(self.node.unlock())


Let us start by analysing the Initialisation

The initialisation and automation takes the seven lines of code and automates them

In [ ]:
from tdmclient import ClientAsync

client = ClientAsync()
node = aw(client.wait_for_node())
aw(node.lock())
# Code to desactivate leds on start
aw(node.compile("""call leds.prox.v(0, 0)\ncall leds.prox.h(0, 0, 0, 0, 0, 0, 0, 0)\ncall leds.temperature(0, 0)\ncall leds.top(0, 0, 0)\ncall leds.circle(0, 0, 0, 0, 0, 0, 0, 0)\ncall leds.buttons(0, 0, 0, 0)\ncall leds.sound(0)\ncall leds.rc(0)"""))

aw(node.unlock())
        

Thanks to the init function there will now be the possibility conserve the client and the node in the class which enables us to have very lean and encapsulated code only having to write

In [ ]:
from dependencies.AsyncClientInterface import AsyncClientInterface

AsyncClient = AsyncClientInterface()

Let us now move on to the function that are important the project the first being set_motors

Through this class I aimed to control the motors through a very permissive method. I had to work around the problem of not being able to override functions in python I therefore used args and kwargs and some logic to extract the left and right motor value through args and kwargs and also made some error management logic.

In [ ]:
def set_motors(self, *args, **kwargs) -> None:
        """
        A flexible function that lets you input the speed of the motors

        :param args: left_motor then right_motor
        :type args: int
        :param kwargs: can put in right_motor or left_motor value in directly or put in array with length 2
        :type kwargs: int or list[int, int]

        :return: None
        """
        if len(args) == 2:
            aw(self.node.set_variables({
                self.LEFT_MOTOR: [args[0]],
                self.RIGHT_MOTOR: [args[1]]
            }))

        elif len(args) == 1:
            aw(self.node.set_variables({
                self.LEFT_MOTOR: [args[0]]
            }))

        if 'left_motor' in kwargs:
            if isinstance(kwargs['left_motor'], int):
                aw(self.node.set_variables({
                    self.LEFT_MOTOR: [kwargs['left_motor']]
                }))
            else:
                raise TypeError("value attributed to left_motor was not int")

        if 'right_motor' in kwargs:
            if isinstance(kwargs['right_motor'], int):
                aw(self.node.set_variables({
                    self.RIGHT_MOTOR: [kwargs['right_motor']]
                }))
            else:
                raise TypeError("value attributed to right_motor was not int")
        if 'motor' in kwargs:
            if isinstance(kwargs['motor'], list) and len(kwargs['motor']) == 2:
                aw(self.node.set_variables({
                    self.LEFT_MOTOR: [int(kwargs['motor'][0])],
                    self.RIGHT_MOTOR: [int(kwargs['motor'][1])]
                }))
            else:
                raise TypeError("either array too long or too short or did not respect type list")


We can notice that in this code we use variables such as self.LEFT_MOTOR and self.RIGHT_MOTOR which are set motor.left.target and motor.right.target respectively and are put under the form of variables to avoid magic strings and make code more readable

This means that one can call the function in different ways

In [ ]:
AsyncClient.set_motors(50, 50)
AsyncClient.set_motors(left_motor=100, right_motor=100)
AsyncClientInterface.set_motors(motor=[100, 100])

I will skip over the functions that have not been used for the project I decided to keep them because they are a nice addition and could have potentially been useful

Now onto get_sensor which is a polyvalent function that enables us to get the value of any sensor

In [ ]:
    def get_sensor(self, sensor: str, calibrated: bool = False) -> list[int] | int | bool:
        """
        This is a function that returns the value of the sensor it accepts only a string and uses a switch case method


        :param sensor : Accepts  a limited number of strings\
            - 'prox_del' : this presents the array of prox ground delta with the option of calibration\
            - 'prox_ref' : this presents the array of prox ground reflected with the option of calibration\
            - self.CONSTANT: this is presented in the example in the description

        :type sensor: str

        :return: returns the value or array of a sensor, or returns False if invalid string is passed as argument
        :rtype: list[int] or int or bool or list[bool]

        Example:

        >>>Client = AsyncClientInterface(1.35,1.35)

        >>>print(Client.get_sensor(Client.PROX_GROUND_DELTA))
        """
        if sensor == "prox_del":
            aw(self.node.wait_for_variables({self.PROX_GROUND_DELTA}))
            if calibrated:
                calibrated_sensor_2 = int(list(self.node.v.prox.ground.delta)[1]) * self._delta_calib
                return [int(list(self.node.v.prox.ground.delta)[0]), int(calibrated_sensor_2)].copy()
            else:
                return list(self.node.v.prox.ground.delta).copy()

        elif sensor == "prox_ref":
            aw(self.node.wait_for_variables({self.PROX_GROUND_REFLECTED}))
            if calibrated:
                calibrated_sensor_2 = int(list(self.node.v.prox.ground.reflected)[1]) * self._refl_calib
                return [int(list(self.node.v.prox.ground.reflected)[0]), int(calibrated_sensor_2)].copy()
            else:
                return list(self.node.v.prox.ground.reflected).copy()

        else:
            aw(self.node.wait_for_variables({sensor}))
            attributes = sensor.split(".")
            node_with_attr = getattr(self.node, "v")
            for attribute in attributes:
                node_with_attr = getattr(node_with_attr, attribute)

            try:
                if not type(node_with_attr) == int:
                    return list(node_with_attr)
                else:
                    return node_with_attr
            except AttributeError:
                raise InvalidArgumentError("attribute was not found due to wrong input being put in")


there are two sensor which needed calibration on the thymio so I added them as a special condition with prox_del and prox_refl so they follow textbook code to get the values of sensors from the node. They were not used for the project so I will skip over this part of the code

Now the most interesting part of the code is this snippet where I split the incoming string use getattr function in order to get the values of the sensors 

In the upcoming section of code I split the code at full stops present in the incoming string and then use getattr function imported from python in order to if we call the function with "motor.left.speed" the code will emulate fetching node.v.motor.left.speed  

In [ ]:
else:
    aw(self.node.wait_for_variables({sensor}))
    attributes = sensor.split(".")
    node_with_attr = getattr(self.node, "v")
    for attribute in attributes:
        node_with_attr = getattr(node_with_attr, attribute)

    try:
        if not type(node_with_attr) == int:
            return list(node_with_attr)
        else:
            return node_with_attr
    except AttributeError:
        raise InvalidArgumentError("attribute was not found due to wrong input being put in")


this enables us to write lean code such as

In [ ]:
AsyncClient.get_sensor(AsyncClient.LEFT_SPEED)

Then I put in set_led enables to set the leds and checks for 3 values.

In [ ]:
    def set_led(self, sensor: str, value: int | list[int]) -> None:
        """
        A function that enables you to set sensors through the node.set_variables method the leds expect values between 0 and 32
        :param sensor: input your sensor
            - 'leds_top': expects three rgb values

        :param value: a list or an array containing the values you want to output
        """

        if sensor == "leds_top":
            if isinstance(value, list) and len(value) == 3:
                aw(self.node.wait_for_variables({"leds.top"}))
                for index, rgb in enumerate(value):
                    self.node.v.leds.top[index] = rgb
            else:
                raise TypeError("either did not input list or list was not of right length")

        self.node.flush()


We can then set the upper leds to blue with this bit of code

In [ ]:
AsyncClient.set_leds("leds_top",[0,0, 32])

### The ThymioRobot class